# The Purpose of This Notebook Will be to highlight all the Steps Required to Make the Recipe 2 Style App

# Data Collection-Selenium & Yummly API

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

In [ ]:
def inf_scroll(scrolls=1000):
    
    '''This function imitiates a user infinitely scrolling to the bottom of a webpage'''

    loop_counter = 0
    #driver.get("http://www.museodelprado.es/coleccion/obras-de-arte")
    #time.sleep(5)
    while loop_counter < scrolls:
        #     driver.execute_script("window.scrollTo(0, 300)")
        #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        scroll = driver.find_element_by_tag_name('body')
        scroll.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        if loop_counter%100 == 0:
            print(loop_counter)
        loop_counter+=1

In [ ]:
driver.get('https://www.yummly.com/recipes?sortBy=relevance&allowedCuisine=cuisine%5Ecuisine-barbecue-bbq') 
#do this for all desired cuisine types 

In [ ]:
inf_scroll(2000)

In [ ]:
element=driver.find_elements_by_xpath('//div[@class="RecipeContainer"]/div/a')
barbeque_recipes=[x.get_attribute('href') for x in element ]    
import pickle
with open('barbeque.pickle', 'wb') as f:
    pickle.dump(barbeque_recipes, f)
    
###repeat for all the necessary regional cuisine styles

## Get Recipe Data From Yummly API


In [1]:
cuisines= [
    'barbeque',
    'british',
    'cajun',
    'chinese',
    'cuban',
    'french',
    'german',
    'greek',
    'hawaiian',
    'hungarian',
    'indian',
    'irish',
    'italian',
    'japanese',
    'keto',
    'mexican',
    'moroccan',
    'paleo',
    'portuguese',
    'southwestern',
    'spanish',
    'swedish',
    'thai',
    'vegan',
    'vietnamese'
            ]

In [ ]:
recipes=[]
for item in cuisines: ###load all the recipes from the pickles
    with open(item+'.pickle','rb') as p:
        l=pickle.load(p)
        print(item)
        recipes.append(l)


In [ ]:
yummly_recipes=[] #make the recipe urls amenable to the yummly api
for item in recipes:
    for x in item:
        yummly_recipes.append(x.replace('https://www.yummly.com/recipe/', ''))B

In [ ]:
yummly_recipes.remove('https://www.yummly.com/page/hellmanns') #weird that it grabbed this one 

In [ ]:
for link in yummly_recipes: #grab info from the yummly api
    try:
        response=requests.get('http://api.yummly.com/v1/api/recipe/'+link+'?_app_id=MYAPPID&_app_key=MYAPPKEY')
        yummly_responses.append(response.json())
        print(link)
    except:
        pass

In [ ]:
with open('yummly_responses.pickle', 'wb') as f:
    pickle.dump(yummly_responses, f)

In [ ]:
yummly_data=[] #Get all the data we want from the Yummly API
for response in yummly_responses:
    recipe_dict={}
    try:
        recipe_dict['name']=response['id']
    except:
        pass
    try:
        recipe_dict['yummly_url']=response['attribution']['url']
    except:
        pass
    try:
        recipe_dict['recipe_url']=response['source']['sourceRecipeUrl']
    except:
        pass
    try:
        recipe_dict['cuisine']=response['attributes']['cuisine'][0]
    except:
        pass
    try:
        recipe_dict['cook_time']=response['cookTimeInSeconds']
    except:
        pass
    try:
        recipe_dict['prep_time']=response['prepTimeInSeconds']
    except:
        pass
    try:
        recipe_dict['rating']=response['rating']
    except:
        pass
    try:
        recipe_dict['ingredients']=response['ingredientLines']
    except:
        pass
    try:
        recipe_dict['image']=response['images'][0]['hostedLargeUrl']
    except:
        pass
    try:
        for item in response['nutritionEstimates']:
            if item['attribute']=='ENERC_KCAL':
                 recipe_dict['calories']=item['value']
    except:
        pass
    try:
        for x in response['nutritionEstimates']:
            if x['attribute']=='PROCNT':
                recipe_dict['protein']=x['value']
    except:
        pass
    try:
         for x in response['nutritionEstimates']:
            if x['attribute']=='CHOCDF':
                recipe_dict['carbs']=x['value']
    except:
        pass
    try:
        for x in response['nutritionEstimates']:
            if x['attribute']=='FAT':
                recipe_dict['fat']=x['value']
    except:
        pass
    try:
        for x in response['nutritionEstimates']:
            if x['attribute']=='SUGAR':
                recipe_dict['sugar']=x['value']
    except:
        pass
    try:
        for x in response['nutritionEstimates']:
            if x['attribute']=='NA':
                recipe_dict['sodium']=x['value']
    except:
        pass
    try:
        for x in response['nutritionEstimates']:
            if x['attribute']=='FIBTG':
                recipe_dict['fiber']=x['value']
    except:
        pass
    try:
        recipe_dict['bitter']=response['flavors']['Bitter']
    except:
        pass
    try:
        recipe_dict['meaty']=response['flavors']['Meaty']
    except:
        pass
    try:
        recipe_dict['piquant']=response['flavors']['Piquant']
    except:
        pass
    try:
        recipe_dict['salty']=response['flavors']['Salty']
    except:
        pass
    try:
        recipe_dict['sour']=response['flavors']['Sour']
    except:
        pass
    try:
        recipe_dict['sweet']=response['flavors']['Sweet']
    except:
        pass
    yummly_data.append(recipe_dict)
    

In [ ]:
df=pd.DataFrame(yummly_data)

In [ ]:
#df.to_pickle('yummly_dataframe.pickle')

# Yummly Data Cleaning 

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_pickle('cuisine_data.pickle')

In [ ]:
df.drop_duplicates(subset='yummly_url', inplace=True) #get rid of duplicate columns
df.dropna(subset=['bitter'], inplace=True) #get rid of any recipes that don't have recipe data 
df.drop('cuisine_x',axis=1,inplace=True) #we want only our custom cuisine type
df.drop(['cook_time','prep_time'], axis=1, inplace=True) #too many missing values


In [ ]:
import re #now to clean up some strings 
df['name_clean']=df.apply(lambda x: re.sub(r'[0-9]+', '', x['name']),axis=1) ###get rid of recipe id 
df['name_clean']=df.apply(lambda x: re.sub(r'-+', ' ',x['name_clean']),axis=1) ### clean up dashes


In [ ]:
### now lets replace NaN images with random stock images 
image_url='http://4.bp.blogspot.com/-1PPIpuTPnPY/UCudijf1DPI/AAAAAAAABgY/Ohzq0co9uyk/s1600/generic.jpg'
df['image'].fillna(value=image_url, inplace=True)

In [ ]:
### replace null nutrition information with mean of that column
df['calories'].fillna(value=df['calories'].mean(), inplace=True)

df['carbs'].fillna(value=df['carbs'].mean(), inplace=True)

df['fat'].fillna(value=df['fiber'].mean(), inplace=True)

df['fiber'].fillna(value=df['fiber'].mean(), inplace=True)

df['protein'].fillna(value=df['protein'].mean(), inplace=True)

df['sodium'].fillna(value=df['sodium'].mean(), inplace=True)

df['sugar'].fillna(value=df['sugar'].mean(), inplace=True)

In [ ]:
#Now we need to clean up the ingredients, (get rid of 2 cups, 1 oz shredded, etc)
#Luckily Someone has made a handy table already!!!
food_stop_words='''1
2
3
4
5
6
7
8
9
10
tsp
t
a
oz
with
extra
tbs
i
e
o
if
on
at
tb
s
fra
de
g
kg
lb
powdered
of
light
ground
sauce
fresh
aged
quality
scant
use
generous
pastry
bag
diameter
tip
duty
plus
stuffed
slivered
blanched
unsalted
flakes
thaw
thawed
plumped
one
two
three
four
five
six
seven
eight
nine
ten
cup
teaspoon
cups
heavy
duty
stand
mixer
then
any
reserved
for
another
measured
measure
about
preferably
fine
quality
well
roughly
rough
do
not
additional
pan
inch_diameter
inch
diameter
reserved
ounce
chopped
tablespoons
ground
and
to
taste
fresh
teaspoons
or
can
sliced
powder
pound
all
package
minced
diced
into
chunks
cup
dry
uncooked
peeled
and
the
in
cubes
inch
to
taste
chopped
cups
tsp
tablespoon
tbsp
quart
sliced
into
torn
stemmed
minced
cracked
rings
whole
strips
bunch
sprigs
fresh
cut
pinch
pound
finely
seeded
as
needed
need
chunks
juiced
soaked
soak
overnight
low
optional
piece
head
large
small
cored
medium
such
cover
pitted
halved
quartered
package
packages
grated
separated
florets
softened
half
medium
freshly
of
halves
beaten
cooked
leaves
needed
pieces
crushed
extra
pitted
flavored
unsweetened
prepared
light
jar
black
red
ones
branches
plain
stick
pot
cube
ripe
baby
vac
semi
gem
paste
chunks
quartered
seasoned
sweetened
thin
flaked
rolled
canned
mashed
garnish
french
evaporated
liquid
lengthwise
sifted
wedges
puree
pure
pint
quarts
squares
undrained
chilled
raw
separated
c
zested
hard
frozen
drained
rinsed
squeezed
welfare
x
g
ml
sticks
higher
spatchcocked
free
range
bite
sized
bone
boneless
breast
bunches
cans
coarsely
container
cooking
cream
crumbled
crumbs
cubed
dash
dashes
decoration
degrees
divided
dredging
dried
dusting
envelope
extract
f
filling
fine
finely
fluid
frying
heavy
hot
hulled
instant
jars
juice
lean
lightly
links
matchstick
matchsticks
meat
minced
melted
minutes
mix
mre
ounces
packed
pounds
or
quick
recipe
refrigerated
removed
room
sargento
scrubbed
seasoning
seed
sharp
size
skin
skinless
slices
sodium
sour
stalk
stock
suce
sweet
temperature
thick
thinly
toothpicks
traditional
trimmed
vegetable
warmed
*
oz
,'''


In [ ]:
food_stop_words=food_stop_words.split('\n')
def lower_case(row):
    return [x.lower() for x in row]
        

In [ ]:
df['ingredients']=df['ingredients'].apply(lower_case) #make the ingredients lower case to remove with stop words

In [ ]:
a=[]
for ingredient in df['ingredients'][0]:
    b=[]
    temp_list=ingredient.split()
    for item in temp_list:
        if item not in food_stop_words:
            b.append(item)
            c=' '.join(b)
    a.append(c)

In [ ]:
def remove_stop(row):
    '''This function removes volumes and measurements and other miscellaneous stop words from the ingredient list'''
    
    import re
    cleaned_ingredients=[]
    
    for ingredient in row:
        temp_ingredient=[]
        temp_list=ingredient.split()
        
        for item in temp_list:
            if item not in food_stop_words and '(' not in item and ')' not in item and '1' not in item and '/' not in item:
                item=re.sub(r'[0-9]+', '', item)
                if item.isalpha:
                    temp_ingredient.append(item)
        
        cleaned_ingredients.append(' '.join(temp_ingredient))
        
    return cleaned_ingredients

In [ ]:
df['ingredients']=df['ingredients'].apply(remove_stop)

In [ ]:
df.to_pickle('clean_cuisine_df.pickle')


## Now we need to get a list of Standard food ingredients from the BBCs website


In [ ]:
#Here is the scrapy code that was used 

In [ ]:
import scrapy


class ExampleSpider(scrapy.Spider):
    name = 'bbc_ingredients_scraper'

    custom_settings = {
        "DOWNLOAD_DELAY": 3,
        "CONCURENT_REQUESTS_PER_DOMAIN": 3,
        "HTTPCACHE_ENABLED": True,
        'ROBOTSTXT_OBEY': False
    }

    start_urls = ['https://www.bbc.com/food/ingredients']

    def parse(self, response):

        for link in response.xpath('//ol[@class="resource-nav"]/li/a/@href').extract()[:25]:

            yield scrapy.Request(
                url='https://www.bbc.com'+link,
                callback=self.parse_bbc
            )

    def parse_bbc(self, response):

        ingredients = response.xpath('//li[@class="resource food"]/@id').extract()

        yield {
        'ingredients':ingredients
        }


In [ ]:
df2=pd.read_json('bbc_ingredient/bbc_ingredients.json') #read in a list of bbc ingredients to standardize against

In [ ]:
bbc_ingredients=[x for x in df2['ingredients']]

In [ ]:
bbc=[]
for item in bbc_ingredients:
    for ingred in item:
        bbc.append(ingred)

In [ ]:
corrected=[]
for item in bbc:
    if '_' in item:
        item=item.replace('_',' ')
        corrected.append(item)
    if 'coriander' in item:
        item=item.replace('coriander','cilantro')
        corrected.append(item)
    if 'aubergine' in item:
        item=item.replace('aubergine','eggplant')
    else:
        corrected.append(item)

In [ ]:
bbc_ingredients=set(corrected)

In [ ]:
df=pd.read_pickle('clean_cuisine_df.pickle')

In [ ]:
def match_ingredient(row):
    corrected_ingredients=[]
    for item in row:
        corrected_ingredient=get_closest_match(item,bbc_ingredients)
        if corrected_ingredient == None:
            corrected_ingredient='salt'
        corrected_ingredients.append(corrected_ingredient)
    return corrected_ingredients
        

In [ ]:
def correct_None(row):
    corrected_ingredients=[]
    for item in row:
        if item == None:
            corrected_ingredient='salt'
        else:
            corrected_ingredient=item
        corrected_ingredients.append(corrected_ingredient)
    return corrected_ingredients

In [ ]:
df['ingredients']=df['ingredients'].apply(match_ingredient)
df['ingredients']=df['ingredients'].apply(correct_None)

In [ ]:
df.to_pickle('clean_cuisine_df.pickle')

# Now we need to Build out the Word2Vec Model and t-SNE plot

In [ ]:
df=pd.read_pickle('clean_cuisine_df.pickle')

In [ ]:
ingredients=[x for x in df['ingredients']]bb

In [ ]:
import gensim  # using skip-gram
model = gensim.models.Word2Vec(ingredients, size=300, window=4, min_count=1, workers=2) ##set parameters for word2vec

In [ ]:
model.most_similar('cheddar cheese', topn=10)

In [ ]:
def get_w2v_score(row):
    '''This function will take a recipe and return the average of all ingredient vecotrs'''
    import numpy as np
    
    ingredient_scores=[]
    
    for item in row:
        ingredient_score=np.mean(model[item])
        ingredient_scores.append(ingredient_score)
    
    return np.mean(ingredient_scores)

In [ ]:
df['recipe_score']=df['ingredients'].apply(get_w2v_score)

In [ ]:
df_ing=df[['ingredients','cuisine_y']]

In [ ]:
def get_recipe_vec(l):
    
    count=0
    matrix_addition=np.zeros(300)
    
    for item in l:
        count+=1
        
        matrix_addition+=np.array(model[item])
        
    return (matrix_addition/count)
    

In [ ]:
df_ing['recipe_vec']=df_ing['ingredients'].apply(get_recipe_vec)

In [ ]:
x=[]
for item in df_ing['recipe_vec'].values:
    x.append(item)

In [ ]:
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300) #use t-SNE to reduce to 2 dimensions

In [ ]:
results=tsne.fit_transform(x)

In [ ]:
df_ing['x_tsne']=results[:,0]

In [ ]:
df_ing['y_tsne']=results[:,1]

In [ ]:
#Separate out the output into x,y components
x = df_ing['x_tsne']
y = df_ing['y_tsne'

topic = df_ing['cuisine_y'].unique()

           #Select your color map, nipy_spectral is great for getting differentiated and readable colors
cmap = matplotlib.cm.get_cmap('nipy_spectral')
cmapScale = int(len(topics))

#Rather than manually setting the sizes for everything, use the scale to keep everything in proportion
figScale = 3

plt.figure(figsize=(10*figScale, 10*figScale),facecolor='white')

#For every topic, plot the points where that topic is the max and
#find the topic center by taking the median location of all points associated
#with the topic; draw a text box over it
labels = [x for x in topics]
for i in range(labels):
    boolArr = np.array(labels) == i
    plt.scatter(x[boolArr],y[boolArr],c=cmap(i*cmapScale),s=100)
    x_avg = np.median(x[boolArr])
    y_avg = np.median(y[boolArr])
    plt.annotate(f'#{i+1}: {topics[i][0]}',
                 xy=(x_avg, y_avg),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='center',
                 va='center',
                 fontsize=30,
                 bbox=dict(boxstyle="round", fc="whitesmoke",alpha=0.9))
plt.axis('off')
plt.title('t-SNE Plot for Job Descriptions with 15 Topics',fontsize=10*figScale)
plt.show()

# Now to Build the Final Recommendation Function

In [ ]:
df=pd.read_pickle('final_recipes_df.pickle')

## Building Similarity Matrix

In [ ]:
df.set_index('name_clean', inplace=True)
flavor_profile_df=df[['bitter','meaty','piquant','salty','sour','sweet','recipe_score']]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dists=cosine_similarity(flavor_profile_df)
sims_df2=pd.DataFrame(dists)

sims_df2.index=flavor_profile_df.index

sims_df2.columns=flavor_profile_df.index

sims_df2.to_hdf('sims_df2.hdf','sims_df',mode='w')



sims_df2 = pd.read_hdf('sims_df2.hdf','sims_df') #Need to use hdf format because file is too large for pickle

## Now we will make a function to return the closest recipe matches for a particular cuisine

In [ ]:
def find_nearest_recipes(recipe, cuisine ,number=10):
    
    import pandas as pd

    
    import jellyfish

    def get_closest_match(x, list_strings):

        best_score= 0
        best_match=None

        for title in list_strings:

            current_score=jellyfish.jaro_winkler(x, title)

            if current_score > best_score:
                best_score = current_score
                best_match = title 

        return best_match
    
    recipe = get_closest_match(recipe, recipes_df.index)
    
    bool_array=[]
    
    for i in recipes_df['cuisine_y']:
        
        try:
            bool_array.append(i==cuisine)
        
        except:
            bool_array.append(None)
            
    bool_dict=dict(zip(recipes_df.index,bool_array))
    
    bool_df=pd.Series(bool_dict).to_frame()
    
    matching_recipes=sims_df.join(bool_df[bool_df[0]==True], how='inner')
    
    final_recipes=recipes_df.join(matching_recipes[recipe].sort_values()[1:number+1], how='inner')
    
    return final_recipes


In [ ]:
def pack_recipes(df):
        '''Will take resuls of finding nearest bars and return a list of bar dictionaries containing relevant info'''
        
        import operator
        
        recipes=[]

        for index, row in df.iterrows():
            a=row.index
            a={}
            a['name']=row.index.capitalize()
            a['image']=row['image']
            a['meaty']=row['meaty']
            a['piquant']=row['piquant']
            a['salty']=row['salty']
            a['sweet']=row['swett']
            a['sour']=row['sour']
            a['yummly_url']=row['yummly_url']
            a['recipe_url']=row['recipe_url']
            a['cuisine']=row['cuisine']
            a['protein']=row['protein']
            a['carbs']=row['carbs']
            a['fat']=row['fat']
            a['sugar']=row['sugar']
            a['sodium']=row['sodium']
            a['calories']=row['calories']

            
            recipes.append(a)

        
        return bars



In [ ]:
### This is the final function for recommending similar cuisines based on cosine similarity of flavor vectors

In [ ]:
def send_recipes(recipe, cuisine, number=9):    
    
    
    def find_nearest_recipes(recipe, cuisine ,number=10):

        import pandas as pd


        import jellyfish

        def get_closest_match(x, list_strings):

            best_score= 0
            best_match=None

            for title in list_strings:

                current_score=jellyfish.jaro_winkler(x, title)

                if current_score > best_score:
                    best_score = current_score
                    best_match = title 

            return best_match

        recipe = get_closest_match(recipe, recipes_df.index)


        matching_recipes=sims_df.join(recipes_df[recipes_df['cuisine_y']==cuisine], how='inner')

        final_recipes=recipes_df.join(matching_recipes[recipe].sort_values()[1:number+1], how='inner')

        return final_recipes



    def pack_recipes(df):
            '''Will take resuls of finding nearest bars and return a list of bar dictionaries containing relevant info'''

            import operator

            recipes=[]

            for index, row in df.iterrows():
                a=row.index
                a={}
                a['name']=row['full_name']
                a['image']=row['image']
                a['meaty']=row['meaty']
                a['piquant']=row['piquant']
                a['salty']=row['salty']
                a['sweet']=row['sweet']
                a['sour']=row['sour']
                a['yummly_url']=row['yummly_url']
                a['recipe_url']=row['recipe_url']
                a['cuisine']=row['cuisine_y']
                a['protein']=row['protein']
                a['carbs']=row['carbs']
                a['fat']=row['fat']
                a['sugar']=row['sugar']
                a['sodium']=row['sodium']
                a['calories']=row['calories']


                recipes.append(a)


            return recipes
        
    return pack_recipes(find_nearest_recipes(recipe, cuisine, number))